In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import time
start_time = time.time()

In [2]:
def find_background_color(image):
    # Define the four corner regions
    corner_regions = [
        image[:1, :1],
        image[:1, -1:],
        image[-1:, :1],
        image[-1:, -1:]
    ]

    # Calculate the mean color for each corner region
    mean_colors = []
    for region in corner_regions:
        mean_color = np.mean(region, axis=(0, 1))
        mean_colors.append(mean_color)

    # Find the maximum occurring color among the corner colors
    corner_color_counts = {}
    for color in mean_colors:
        color_key = tuple(color)
        if color_key in corner_color_counts:
            corner_color_counts[color_key] += 1
        else:
            corner_color_counts[color_key] = 1

    max_occuring_color = max(corner_color_counts, key=corner_color_counts.get)

    # Convert the mean color to the HSV color space
    max_occuring_color_hsv = cv2.cvtColor(np.uint8([[max_occuring_color]]), cv2.COLOR_BGR2HSV)[0][0]

    return max_occuring_color_hsv

In [3]:
def process_image(image, background_color):
    # Define the exact range for the background color (tolerance = 0)
    lower_range = np.array([background_color[0], background_color[1], background_color[2]], dtype=np.uint8)
    upper_range = np.array([background_color[0], background_color[1], background_color[2]], dtype=np.uint8)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a binary mask by thresholding the image based on the defined range
    mask = cv2.inRange(hsv_image, lower_range, upper_range)

    # Dilate the mask to expand the foreground regions and remove the background
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilated = cv2.dilate(mask, kernel, iterations=2)

    # Apply Canny edge detection to detect edges
    edges = cv2.Canny(dilated, 100, 200)

    # Apply morphological dilation to make the edges thicker
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    thick_edges = cv2.dilate(edges, kernel, iterations=3)

    # Invert the colors of the image
    inverted_edges = cv2.bitwise_not(thick_edges)

    # Define the coordinates for the segment extraction
    start_x = 350
    end_x = 450
    start_y = 0
    end_y = 100

    # Extract the segment from the image
    segment = inverted_edges[start_y:end_y, start_x:end_x]

    return segment

In [4]:
def image_processing(i):
    # Load the captcha image
    image = cv2.imread(f"train/{i}.png")

    # Find background color
    background_color = find_background_color(image)

    # Process the image
    segment = process_image(image, background_color)

    # Create the output folder if it doesn't exist
    output_folder = "output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save the resulting image with white background
    cv2.imwrite(f"{output_folder}/final_{i}.png", segment)

# Process the images and save the output
for i in range(2000):
    image_processing(i)

In [5]:
file = open( "train/labels.txt", "r" )
label_train = file.read().splitlines()
file.close()
label_train = np.array(label_train)
data_dir = 'output/'

In [6]:
image_size = (100, 100)
# Load the images and convert them to grayscale
images = []
for i in range(2000):
    image_path = f'{data_dir}final_{i}.png'
    image = Image.open(image_path).convert("L")
    image = np.array(image.resize(image_size)).flatten()  # Resize to specified size and flatten
    images.append(image)
# Load the labels from the file
with open("train/labels.txt", "r") as file:
    labels = file.read().splitlines()

# Verify the number of labels matches the number of images
if len(labels) != len(images):
    raise ValueError("Number of labels does not match number of images")
# Step 2: Split the dataset into training and testing sets
# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


img2 = images[0:1900]
label2_train = label_train[0:1900]
img2_test = images[1900:2000]
labels_test = label_train[1900:2000]


clf = SVC(kernel="linear")
clf.fit(img2, label2_train)

# Step 4: Predict the labels for the test set
y_pred = clf.predict(img2_test)

# Step 5: Evaluate the accuracy of the classifier
accuracy = accuracy_score(labels_test, y_pred)
print("Accuracy:", accuracy)


 # Step 4: Make predictions on the testing set
y_pred = clf.predict(X_test)

# # Step 5: Evaluate the model
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0
Accuracy: 1.0


In [7]:
k = 5  # Number of neighbors
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(img2,label2_train)

# Step 4: Predict the labels for the test set
y_pred = knn.predict(img2_test)

# Step 5: Evaluate the accuracy of the classifier
accuracy = accuracy_score(labels_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

# Train the decision tree classifier
dt.fit(img2, label2_train)

# Make predictions on the test set
y_pred = dt.predict(img2_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(labels_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.99


In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

# Train the decision tree classifier
lr.fit(img2, label2_train)

# Make predictions on the test set
y_pred = lr.predict(img2_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(labels_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [10]:
import pickle

# Assuming you have a trained model called "model"
model = lr  # Your trained model object

# Define the file path to store the model
model_file = 'model_lr.pkl'

# Save the model to a file
with open(model_file, 'wb') as file:
    pickle.dump(model, file)

In [11]:
# Load the labels from the file
with open("train/labels.txt", "r") as file:
    labels = file.read().splitlines()

# Verify the number of labels matches the number of images
if len(labels) != 2000:
    raise ValueError("Number of labels does not match number of images")

data_dir = 'output/'
image_size = (100, 100)

# Load the images and convert them to grayscale
images = []
for i in range(2000):
    image_path = f'{data_dir}final_{i}.png'
    image = Image.open(image_path).convert("L")
    image = np.array(image.resize(image_size)).flatten()  # Resize to specified size and flatten
    images.append(image)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Support Vector Machine Classifier
clf = SVC(kernel="linear")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("SVM Accuracy:", accuracy)

# K-Nearest Neighbors Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("KNN Accuracy:", accuracy)

# Decision Tree Classifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy:", accuracy)

# Logistic Regression Classifier
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy:", accuracy)

# Save Logistic Regression model to a file
model_file = 'model_lr.pkl'
with open(model_file, 'wb') as file:
    pickle.dump(lr, file)


SVM Accuracy: 1.0
KNN Accuracy: 1.0
Decision Tree Accuracy: 0.99
Logistic Regression Accuracy: 1.0


In [12]:
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Execution time: 45.0737247467041 seconds
